In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier #knn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
data= pd.read_csv("C:\\Users\\MASSIVE\\Price-Predictor-ML-\\final.csv")

In [ ]:
print(data.columns)
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
df_sans_redondance = data.drop_duplicates(subset=['title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'option0', 'option1', 'option2',
       'option3', 'option4', 'option5', 'option6', 'option7', 'option8',
       'option9', 'option10', 'option11', 'option12', 'option13'])

In [ ]:
df_sans_redondance.describe()

In [ ]:
copy=df_sans_redondance.copy()

In [ ]:
# Create a new column named 'new_options' and initialize it with 0
copy['options'] = 0

# Iterate over each row
for index, row in copy.iterrows():
    options_count = 0
    # Count options from option0 to option13
    for i in range(14):
        option_col = f'option{i}'
        if not pd.isnull(row[option_col]):
            options_count += 1
    # Store the count of options in the 'new_options' column
    copy.at[index, 'options'] = options_count

In [ ]:
copy.columns

In [ ]:
copy['options'].value_counts

In [ ]:
copy.drop(['option0', 'option1', 'option2',
       'option3', 'option4', 'option5', 'option6', 'option7', 'option8',
       'option9', 'option10', 'option11', 'option12', 'option13'], axis=1,inplace=True)

In [ ]:
copy.columns

In [ ]:
print(copy[['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite']].dtypes)

In [ ]:
copy['specs_kilometrage'] = pd.to_numeric(copy['specs_kilometrage'].str.replace('km', ''), errors='coerce')

In [ ]:
copy['specs_annee'] = pd.to_numeric(copy['specs_annee'],errors='coerce')
copy['price'] = pd.to_numeric(copy['price'],errors='coerce')

In [ ]:
copy['specs_papiers'].value_counts()

In [ ]:
# Check for null values in the 'specs_energie' column
null_count = copy['specs_boite'].isnull().sum()
print("Number of null values:", null_count)

# Display unique values in the 'specs_energie' column
unique_values = copy['specs_boite'].unique()
print("Unique values:", unique_values)


In [ ]:
copy['specs_papiers'] = copy['specs_papiers'].replace({
    'Carte grise / safia': 1,
    'Licence / Délai': 2,
    'Carte jaune': 3,
})

# Display the resulting DataFrame
print(copy['specs_papiers'].value_counts())

In [ ]:
print(copy[['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options']].dtypes)

In [ ]:
label_encoder = LabelEncoder()
copy['specs_couleur_auto'] = label_encoder.fit_transform(copy['specs_couleur_auto'])

# for the marque column:

In [ ]:
copy['specs_marque-voiture'].value_counts()

In [ ]:
# Mapping dictionary to standardize brand names
brand_mapping = {
    'Renault laguna': 'Renault','رونو' : 'Renault','Clio': 'Renault','Kongo': 'Renault','رنوا': 'Renault', 'R4': 'Renault', 'Renu kongo': 'Renault', 'clio 3': 'Renault',
    'Dacia stpway sandero': 'Dacia', 'Fiat une': 'Fiat', 'FIAT TIPO': 'Fiat', 'كليو': 'Renault', 'Ds7': 'DS Automobiles',
    'Picanto': 'Kia', 'Kia  sportage': 'Kia', 'Ikco': 'Other', 'chevroli': 'Chevrolet', 'Cheri': 'Chery', 'Jetour': 'Chery','chiri': 'Chery',
    'ايبيزا 2015 2 بورت': 'Seat', 'Mini': 'BMW','bmw e36': 'BMW', 'Picasso': 'Citroen', 'Shacman': 'Shacman', '308': 'Peugeot',
    'Golf 2': 'Volkswagen', 'Chérie': 'Chery', '0771720850': 'Other', 'شفرولي': 'Chevrolet', 'Creta': 'Hyundai','Mercedes Benz':'Mercedes',
    'Vw': 'Volkswagen', '305': 'Peugeot', 'جريتويل': 'Great Wall', 'Ronaut': 'Renault', 'Rifter': 'Peugeot','Fiat Professional':'Fiat',
    'سيارات أكسدونتي ورفورم': 'Other', 'Ona': 'Other', 'Range Rover evoque': 'Land Rover', 'Citroën berlingot': 'Citroen',
    'Renault compus': 'Renault', 'Voila corsa': 'Opel', 'Nouvelle': 'Other', 'Grand tigre': 'Other', 'RUNUT': 'Renault',
    'Pugeux': 'Peugeot', 'SANIYA': 'Other', 'Alto': 'Suzuki', 'Changhe': 'Changhe', 'Corvette': 'Chevrolet', 'Siat': 'SEAT',
    '206': 'Peugeot', 'Huanghai': 'Huanghai', 'Symbole': 'Other', 'Clio 2': 'Renault', 'Leon': 'SEAT', 'Maserati': 'Maserati',
    'Tesla': 'Tesla', 'VMS': 'Other', 'Foryota': 'Other', 'Saipa': 'Saipa', 'Smart': 'Smart', 'Jetour': 'Jetour','Lexus':'Toyota',
    'سيات': 'SEAT', 'Lexus': 'Lexus', 'Lancia': 'Lancia', 'QQ': 'Nissan', 'Kangoo': 'Renault', 'Range rover': 'Land Rover',
    'Hummer': 'Hummer', 'Changhe': 'Changhe', 'Saipa': 'Saipa', 'Maruti': 'Suzuki', 'Polo': 'Volkswagen', 'Chrysler': 'Chrysler',
    'Cadillac': 'Cadillac', 'Aston Martin': 'Aston Martin', 'SYM': 'SYM', 'Emgrand': 'Emgrand', 'Ibiza': 'Seat',
    'SEAT': 'Seat', 'Golf': 'Volkswagen', 'Yamaha': 'Yamaha', 'Iran Khodro': 'Iran Khodro', 'Volvo': 'Volvo',
    'Cupra': 'Cupra', 'Dodge': 'Dodge', 'Infiniti': 'Nissan', 'Isuzu': 'Isuzu', 'Sokon': 'Sokon', 'بيجو': 'Peugeot'
}

   
# Replace inconsistent brand names with the correct ones
copy['specs_marque-voiture'].replace(brand_mapping, inplace=True)

In [ ]:
copy = copy[~copy['specs_marque-voiture'].str.contains('Vms')]
copy = copy[~copy['specs_marque-voiture'].str.contains('Sym')]

In [ ]:
from collections import Counter

# Count the occurrences of each brand
brand_counts = Counter(copy['specs_marque-voiture'])

# Identify brands with only one occurrence
single_occurrence_brands = [brand for brand, count in brand_counts.items() if (count <=3 )]

# Map single occurrence brands to 'Other'
for brand in single_occurrence_brands:
    brand_mapping[brand] = 'Other'

# Replace inconsistent brand names with the correct ones
copy['specs_marque-voiture'].replace(brand_mapping, inplace=True)


In [ ]:
# Check the updated value counts
print(copy['specs_marque-voiture'].value_counts())

In [ ]:
value_counts = copy['specs_marque-voiture'].value_counts()

# Sort the value counts from smallest to largest
sorted_value_counts = value_counts.sort_values()

# Print the sorted value counts
print(sorted_value_counts)


In [ ]:
unique_brands = copy['specs_marque-voiture'].unique()

# Print the list of unique brands
print(unique_brands)

# now for the engine

In [ ]:
copy['Engine Capacity'] = copy['specs_car_engine'].str[:3]

In [ ]:
copy['Engine Capacity'] = pd.to_numeric(copy['Engine Capacity'], errors='coerce')
copy.loc[copy['Engine Capacity'] > 26, 'Engine Capacity'] = pd.NA

# Calculate the mean of the "Engine Capacity" column, excluding NaN values
mean_value = copy['Engine Capacity'].mean(skipna=True)

# Fill NaN values in the "Engine Capacity" column with the calculated mean
copy['Engine Capacity'].fillna(mean_value, inplace=True)

In [ ]:
print(copy['Engine Capacity'].value_counts())

In [ ]:
copy.columns

In [ ]:
print(copy[['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options','Engine Capacity']].dtypes)

In [ ]:
copy.describe().T

In [ ]:
column_values = copy['specs_car_engine'].dropna()  # Drop NaN values in the column

# Initialize an empty list to store the unique strings separated by a space
space_separated_strings = []

# Iterate over each value in the column
for value in column_values:
    # Split the value by space
    words = value.split()
    # Check if there are more than one word (separated by space) in the value
    if len(words) > 1:
        # Add each separated string to the list (excluding duplicates)
        for word in words:
            if word not in space_separated_strings:
                space_separated_strings.append(word)

# Print the list of unique str
print(space_separated_strings)

In [ ]:

provided_engine_names = ['HDI', 'TDI', 'VTEC', 'VVTi', 'TSI', 'TFSI', 'ECOBOOST', 'CRDi', 'MPI', 'GDI',
                        'V6', 'V8', 'V12', 'Boxer', 'Wankel', 'Rotary', 'Hybrid', 'Turbo', 'Supercharged',
                        'Diesel', 'Gasoline', 'Electric', 'Petrol', 'CNG', 'LPG', 'EcoDiesel', 'FlexFuel',
                        'Direct Injection', 'Common Rail', 'Variable Valve Timing', 'Variable Valve Lift',
                        'Variable Valve Duration', 'Overhead Cam', 'DOHC', 'SOHC', 'OHV', 'Pushrod',
                        'Hybrid Synergy Drive', 'SkyActiv', 'BlueHDi', 'BlueMotion', 'eTorque', 'EcoBoost',
                        'PureTech', 'EcoTec', 'i-VTEC', 'm-VTEC', 'Dual VVT-i', 'VVT-L', 'Variable Twin Turbo',
                        'ess','dci','DCi','Dci','tce','VVTI','tsi','VVTL','4WD','hDi',"DCI", "MPi", "4WD", "HDI", "TDI", "VTI", "VCDI", "BVA", "VVT", "CDI",
    "Ecomotive", "VVTI", "FAP", "CH", "ESS", "CVVT", "GPL", "CRDI", "DSG",
    "VT", "DDTI", "SR", "DSG6", "SR", "VT", "GDI", "TFSI", "MPI", "HTP",
    "VVTI", "DTR", "MT", "GPL", "CVT", "CGI", "MZR", "DOHC", "D-CVVT",
    "TCI", "GTS", "CVT", "CGI", "CDTI", "DOHC", "D-CVVT", "S", "TCI", "GTS",
    "TDI", "HSE", "GDI", "HLBMT", "SCe", "R-Line", "ACC", "PDC", "S-tronic",
    "SE", "TD", "RS", "FR","DCI", "MPi", "4WD", "HDI", "TDI", "VTI", "VCDI", "BVA", "VVT", "CDI",
    "Ecomotive", "VVTI", "FAP", "CH", "ESS", "CVVT", "GPL", "CRDI", "DSG",
    "VT", "DDTI", "SR", "DSG6", "SR", "VT", "GDI", "TFSI", "MPI", "HTP",
    "VVTI", "DTR", "MT", "GPL", "CVT", "CGI", "MZR", "DOHC", "D-CVVT",
    "TCI", "GTS", "CVT", "CGI", "CDTI", "DOHC", "D-CVVT", "S", "TCI", "GTS",
    "TDI", "HSE", "GDI", "HLBMT", "SCe", "R-Line", "ACC", "PDC", "S-tronic",
    "SE", "TD", "RS", "FR", "ESS","DCI", "MPi", "HDI", "TDI", "VTI", "VCDI", "VVT", "Ecomotive", "VVTI", "FAP",
    "Quattro", "E-XDI", "GTI", "AMG", "4MATIC", "HTP", "VM", "DSG", "S90", "DDTI",
    "GT", "MPI", "S-Tronic", "BVM", "DDI", "CRDI", "DSG6", "SR", "D4D", "PDK",
    "TFSI", "TS", "CGI", "CDI", "CVT", "MZR", "DOHC", "GTS", "V8", "TCI", "TCE",
    "VCT", "GTD", "Om", "CV", "AWD", "STD", "GPL", "VMAX", "DLX", "D4", "Mjet",
    "CD", "DI-D", "VTEC", "XLI", "MHD", "TDI2.0", "GDI", "STT", "SE", "EFI",
    "V6", "Matic", "TDI177", "DTEC", "S&S", "CVVT", "DCI", "TDI", "ess", "TDI",
    "E-DGI", "GDI", "MPI", "ess", "GDI", "MPI", "HTP", "HDI", "HT", "FSI", "GDI",
    "MPI", "GDI", "TDI"]
unique_names = list(set(provided_engine_names))
print(unique_names)
 
names=[ 'AWD', 'BlueHDi', 'CRDi', 'CDTI', 'DCi', 'DDTI', 'DCI', 'D-CVVT', 'DSG6', 'EcoDiesel', 
    'EcoTec', 'EcoBoost', 'Ecomotive', 'E-GDI', 'Electric', 'E-XDI', 'FlexFuel', 'GDI', 'GTD', 
    'GTI', 'Gasoline', 'HTP', 'HLBMT', 'HDI', 'HSE', 'Hybrid', 'Hybrid Synergy Drive', 'MPI', 
    'Matic', 'MHD', 'Mjet', 'MT', 'OHV', 'Om', 'PDK', 'PDC', 'PureTech', 'Petrol', 'Quattro', 
    'R-Line', 'Rotary', 'S-Tronic', 'S&S', 'SCe', 'STD', 'SR', 'SkyActiv', 'S90', 'SOHC', 
    'S-tronic', 'Supercharged', 'TSI', 'TDI2.0', 'TDI177', 'TDI', 'TFSI', 'TCE', 'TCI', 'TS', 
    'VVT-L', 'VVTL', 'Variable Twin Turbo', 'Variable Valve Timing', 'Variable Valve Lift', 
    'Variable Valve Duration', 'VMAX', 'VVTi', 'VVTI', 'VVT', 'VTEC', 'VCT', 'VCDI', 'VTI', 
    'VT', 'VT', 'Wankel', 'XLI', 'ess', 'ess', 'tce', 'tsi', 'ess', 'CDI', 'CVT', 'CVVT', 'CH', 
    'CRDI', 'CD', 'DDI', 'DTR', 'DTEC', 'DOHC', 'DLX', 'Dci', 'Dci', 'Dci', 'D4D', 'D4', 'EFI', 
    'ECOBOOST', 'GPL', 'GTS', 'GT', 'GDI', 'GDI', 'MPi', 'MPi', 'MZR', 'S', 'SE', 'SR', 'TCE', 
    'TD', 'V12', 'V8', 'V6', 'V4', 'V2', 'V', 'DCI', 'DCI', 'DCI', '4WD', '4MATIC', 'STD', 'TS', 
    'SR', 'VT', 'VT', 'VT', 'VCDI', 'VTI', 'VTEC', 'VMAX', 'VVT', 'VVTI', 'VVT-L', 'VVTi', 'VVTI', 
    'VVTL']


In [ ]:
# Assuming engine_types is the list of engine types
sorted_engine_types = sorted(unique_names, key=len, reverse=True)

# Print the sorted list of engine types
print(sorted_engine_types)

In [ ]:
# Assuming 'copy' is the name of your DataFrame and 'specs_car_engine' is the name of the column
# Replace 'specs_car_engine' with the actual name of your column
copy['Engine_Name_Match'] = copy['specs_car_engine'].apply(lambda x: next((engine for engine in sorted_engine_types if isinstance(x, str) and engine.lower() in x.lower()), pd.NA))

# Display the DataFrame with the new column
print(copy)

In [ ]:
print(copy['Engine_Name_Match'].value_counts())

In [ ]:
pd.set_option('display.max_rows', None)

# Print the entire series
print(copy['Engine_Name_Match'].value_counts())

In [ ]:
num_types = copy['Engine_Name_Match'].value_counts().nunique()
print("The number of unique types is:", num_types)

In [ ]:
# Fill missing values with a placeholder string
copy['Engine_Name_Match'] = copy['Engine_Name_Match'].fillna('missing')

# Fit the LabelEncoder to the engine names and transform them into numerical labels
encoded_labels = label_encoder.fit_transform(copy['Engine_Name_Match'])

# Print the encoded labels
print("Encoded labels:", encoded_labels)

# Print the mapping between original labels and encoded labels
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("Label mapping:", label_mapping)


In [ ]:
encoded_labels_int = encoded_labels.astype(int)

# Replace the column in the DataFrame with the integer encoded labels
copy['Engine_Name_Match'] = encoded_labels_int

# Verify the changes
print(copy['Engine_Name_Match'].dtype)

In [ ]:
total_sum = copy['Engine_Name_Match'].value_counts().sum()
print("The sum of counts is:", total_sum)

# now for the horse power

In [ ]:
horsepower_values=[90, 92, 75, 72, 110, 80, 102, 143, 85, 38, 65, 140, 103, 
                   100, 68, 84, 94, 250, 204, 112, 55, 171, 105, 115, 99
                   , 425, 130, 91, 34, 83, 71, 95, 60, 70, 120, 109, 177, 200, 82, 69, 63, 53, 127, 174, 170, 184, 
                   190, 85, 197, 128, 46, 78, 98, 136, 123, 155, 129, 81, 107, 360, 200, 97, 95, 176, 96,
                    245, 210, 88, 125, 86, 113, 52, 163, 58, 140, 245, 160, 300, 114, 64, 93, 117, 185, 62, 200, 104, 54, 138, 122]

In [ ]:
def extract_horsepower(spec):
    # Convert spec to string
    spec_str = str(spec)
    for hp in horsepower_values:
        # Convert hp to string for comparison
        hp_str = str(hp)
        if hp_str in spec_str:
            return hp
    return None

# Create the 'horsepower' column by applying the function to the 'specs_car_engine' column
copy['horsepower'] = copy['specs_car_engine'].apply(extract_horsepower)

# Display the DataFrame with the new 'horsepower' column
print(copy)

In [ ]:
mean_horsepower = copy['horsepower'].mean()

# Fill missing values with the mean
copy['horsepower'].fillna(mean_horsepower, inplace=True)

# Display the DataFrame with the new 'horsepower' column
print(copy)

# now for the kilometrage

In [ ]:
copy.loc[copy['specs_kilometrage'] < 999, 'specs_kilometrage'] *= 1000

In [ ]:
def divide_until_less_than_999999(value):
    value = abs(value)

    # Divide the value by 10 until it becomes less than 999999
    while value >= 999999:
        value /= 10
    return value


In [ ]:
copy['specs_kilometrage']=copy['specs_kilometrage'].apply(divide_until_less_than_999999)

In [ ]:
copy['specs_kilometrage'] = copy['specs_kilometrage'].abs()

In [ ]:
copy.loc[(copy['specs_kilometrage'] >= 1000) & (copy['specs_kilometrage'] <= 9999), 'specs_kilometrage'] = np.nan

In [ ]:
# Display the DataFrame with updated 'specs_kilometrage' column
print(copy['specs_kilometrage'])

In [ ]:
copy.describe()

# LIST OF THINGS TO DO (hadawin fe9t bli kount 9adra nekteb to do list XD)
- for the title i dont think i can do somthing ill just remove it since its combaining between model finition and marque ig
- i wanna first of aaall remove the rows that doesnt contain " price - kilo - anne " these triplet is so important 
- the price
- anne        ☑☑
- engine      ☑     (ill see if ill remove it or not)
- color       ☑     (i wanna optimize it)
- papier      ☑☑
- kilometrage ☑
- marque      
- model
- finition
- energie     ☑
- boite       ☑ 
- options     ☑☑
- 'Engine Capacity','Engine_Name_Match', 'horsepower'

In [ ]:
copy.columns

In [ ]:
print(copy[['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options', 'Engine Capacity',
       'Engine_Name_Match','horsepower']].dtypes)

In [ ]:
print(copy.columns)
copy.isnull().sum()